# Hyperparameter Tuning

As you have learned so far, there are number of optimization algorithms to train deep learning models. Training a complex deep learning model can take hours, days or even weeks. Therefore, random guesses for model's hyperparameters might not be very practical and some deeper knowledge is required. These hyperparameters include but are not limited to learning rate, number of hidden layers, dropout rate, batch size and number of epochs to train for. Notably, not all these hyperparamters contribute in the same way to the model's performance, which makes finding the best configurations of these variables in such high dimensional space a nontrivial challenge (searching is expensive!). In this chapter, we look at different strategies to tackle this searching problem.

Hyperparameters can be categorized into two groups: those used for training and those related to model structure and design.

## Training Hyperparameters

### Learning rate

Gradient descent algorithms multpily the gradient by this scalar known as learning rate to determine the next point in the weight's space. Learning rate is a hyperparameter that controls the step size to move in the direction of lower loss function, with the goal of minimizing it. In most cases, learning rate is manually adjusted during model training. Large learning rates ($\alpha$) make the model learn faster but at the same time it may cause us to miss the minimum loss function and only reach the surrounding of it. In cases where the learning rate is too large, the optimizer overshoots the minimum and the loss updates will lead to divergent behaviours. On the other hand, choosing lower $\alpha$ values gives a better chance of finding the local minima with the trade-off of needing larger number of epochs and more time. 

<img  src=".\loss-lr.gif" width="1000" align="center">
    
Note that we can almost never plot the loss as a function of weight's space (as shown above) and this makes finding the reasonable $\alpha$ tricky.  With a proper constant $\alpha$, the model can be trained to a passable yet still unsatisfactory accuracy, because the constant $\alpha$ can be overlarge, especially in the last few epochs. Alternatively, $\alpha$ can be adaptively adjusted in response to the performance of the model. This is also known as learning rate decay schedule. Some commonly applied decay schedules include linear (step), exponential, polynomial and cyclic. By starting at a larger learning rate, we achieve the rarely discussed benefit of allowing our model to escape the local minima that overfits and find a broader minimum as learning rate decreases over the number of epochs.


<img  src=".\decay.gif" width="750" align="center">
<!-- <video width="320" height="240" autoplay loop muted>
  <source src="movie.mp4" type="video/mp4" />
</video> -->

### Batch Size

The batch size is always a trade-off between computational efficiency and accuracy. By reducing the batch size, you are essentially reducing the number of samples based on which loss is calculated at each training iteration. Since smaller samples can have more variations from one another, they can add mode noise to convergence. Thus, by keeping the batch size small, we are more likely to find a broader local minima. 

As mentioned in the previous section, decaying the learning rate is a common practice in training ML frameworks. It has actually been shown that we can obtain the same benefits and learning curve by scaling up the batch size during training insetead (Don't Decay the Learning Rate, Increase the Batch Size - arXiv)

## Model Design-Related Hyperparameters

### Number of hidden layers

The overall stucture of neural networks are determined based on the number of hidden layers ($d$). Before describing this, let us talk about the traditional disagreement on how the total number of layers are counted. This disagreement centers around whether or not the input layer is counted. There is an argument that suggests it should not be counted since inputs are not active. We go by this convention, which is also recommended in book (Neural smithing: supervised learning in feedforward artificial neural networks). A single-layer can only be used to represent linearly separable functions, in very simple problems. On the other hand, deep learning models with more layers are more likely to capture more complex features are obtain a realative higher accuracy. As a common sense, you can keep adding layers until the test error does not improve anymore.

### Number of nodes in each hidden layer

The number of nodes ($w$) in each layer should be carefully considered to avoid overfitting and underfitting. Small ($w$) may result underfitting as the model lacks complexity. On the other hand, too many nodes (large $w$) can cause overfitting and increase training time. Here are some rule-of-thumbs that can be a good start for tuning the number of nodes (Jeff Heaton. The number of hidden layers, 2017. URL https://www.heatonresearch.com/2017/06/01/hidden-layers.html.):

<ol>
  <li>$w_{input} < w < w_{output}$</li>
  <li>$w = \frac{2}{3}w_{input} + w_{output}$</li>
  <li>$w < 2 w_{output}$</li>
</ol>

### Regularization

Regularization is applied to counteract the additional model complexity that comes as a result of adding more nodes in deep neural networks. The most common regulization methods ($L_1$ and $L_2$) are explained in $REGRESSION CHAPTER$. Hyperparameter $\lambda$ determines the magnitude of the regularization term in the loss function. Too large $\lambda$ pushes the weights closer to zero and oversimplyfies the structure of the deep learning model, where as undersized $\lambda$ is not strong enough to reduce the weights. Thanks to its computational efficiency $L_2$ regularization is more widely used, however, $L_1$ regularization prevails over $L_2$ on sparse properties. A brief comparison between $L_1$ and  $L_2$ are presented in Table (?).

<div align="center">
    
|                         $L_1$                        |                  $L_2$                 |
|:--------------------------------------------------:|:-----------------------------------:|
| Penalizes the sum of the absolute value of weights | Penalizes the sum of square weights |
|          Unable to learn complex patterns          | Able to learn complex data patterns |
|                 Robust to outliers                 |        Not robust to outliers       |
|             Built-in feature selection             |         No feature selection        |
|                  Multiple solution                 |             One solution            |
|                   Sparse solution                  |          Nonsparse solution         |
    
</div>

#### Dropout

Dropout is a regularization technique where random nodes are deactivated and not used during training, which makes the model less sensitive to specific weights of nodes. It is recommended that the probability of node deactivation applied for each weight-updated step is around 20-50%. Too large dropout rates  will oversimplify the model, whereas small values will have minor effect. Moreover, as fewer nodes get updated with dropout, larger learnings rates with decay and a larger momentum can help with the model's performance.

<img  src=".\drop_out.gif" width="250" align="center">

## Hyper-Parameter Optimization

Mathematically, hyperparameter optimization (HPO) is the process of finding the set of hyperparamters to either achieve minimum loss or maximum accuracy of the objective model. The general philosophy is the same for all HPO algorithms: determine which hyperparameters to tune and their corresponding search space, adjust them from coarse to fine and obtain the optimal combination.

The state-of-the-art HPO algorithms are classified into two categories: search algorithms and trial schedulers. In general, serach algorithms are applied for sampling and trial schedulers deal with the early stopping methods for model evaluation.

### Search Algorithms 

#### Grid Search
As long as you have sufficient computational resources, grid search is a straightforward method for HPO. It performs an exhaustive search on the hyperparamters sets defined by the user. Grid search is applicable for cases where we have limited number of hyperparameters with limited search space.

#### Random Search

Random search is a more effective version of grid search but still computationally exhaustive. It performs a randomized search over hyperparamters from certain distributions over possible parameter values. The searching process continues until the desired accuracy is reached or unitl the predetermined computational budget is exhausted. Random search works better than grid serach considering two benefits: First, a budget can be assigned independently based on the distribution of the search space, whereas in grid search the budget for each hyperparameter set is a fixed value. This makes random search perform better than grid search, especially in search patterns where some hyper parameters are not uniformly distributed. Secondly, a larger time consumption of random search quite certainly will lead to a larger probability finding the best hyperparameter set (this is known as Monte Carlo techniques).

#### Bayesian Optimization

### Trial Schedulers

HPO is a time-consuming process and in realistic scenarios, it is necessary to obtain the best hyperparameter with limited available resources. When it comes to training the hyperparamters by hand, by experience we can narrow the search space down, evalaute the model during training and decide whether to stop the training or keep going. An early stopping strategy tries to mimic this behavior and maximize the computational resource budget for promising hyperparameter sets.

#### Successive Halving

Successive Halving (SHA) converts the hyperparameter optimization problem into a non-stochastic best-arm identification and tried to allocate more resources only to those hyperparameter sets that are more promising. In SHA, user defines and fixed budget ($B$) and a fixed number of trials ($n$). The hyperparameter sets are uniformly queried for a portion of the intial budget and the corresponding model performances are evaluated for all tirals. The worst promising half is dropped while the budget is doubled for the other half and this is done successively until one tiral remains. One drawback of SHA is how resources are allocated. There is a trade-off between the total budget ($B$) and number of trials ($n$). If $n$ is too large, each trial may result in premature termination, whereas too small $n$ would not provide enough optional choices.  

Compared with Bayesian optimization, SHA is easier to undestand and it is more computaitonally efficient as it evalaute the intermediate model results and determines whether to terminate it or not. 

#### HyperBand

HyperBand is an extention of SHA that tries to solve the resource allocation problem by considering several possible $n$ values and fixing $B$.

#### Median Stopping

#### Curve Fitting

## Hyper-Parameter Tuning 

Now that we know more about different HPO techniques, let's take a look at one of the examples in Chapter (?). (need to discuss with Andrew which existing example to do hyper-parameter tuning for).